In [35]:
import sys
import os
import shutil
from py import analysis
import matplotlib
# %matplotlib notebook
%load_ext autoreload
%autoreload 2


cwd=os.getcwd()
cwd='/home/agoga/topcon'

folder='/sandbox/'

file='CreateAmorphSiOx.dump'
f=cwd+folder+file
filepath=os.path.join(cwd,f)
f=cwd+folder+"png/"
outFolder=os.path.join(cwd,f)
#filepath="/home/agoga/topcon/sandbox/CreateAmorphSiOx.dump"
print(filepath)
# if os.path.exists(outFolder):
#     shutil.rmtree(outFolder)
# print(outFolder)
# os.makedirs(outFolder)
# coordList=[0,1,2,3,4]
# e=analysis.coordinationTimeseries(os.path.join('/home/agoga/topcon/sandbox/'+f),coordList)
analysis.rdfTimeseries(filepath,[],outFolder)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/agoga/topcon/sandbox/CreateAmorphSiOx.dump


<Figure size 640x480 with 0 Axes>

In [38]:
import imageio
images = []

for filename in sorted(os.listdir(outFolder)):
    f=outFolder+filename
    images.append(imageio.imread(f))
imageio.mimsave(cwd+folder+file+'.mp4', images)

/tmp/ipykernel_1799/2833260775.py:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f))


In [ ]:
import glob
from PIL import Image
import numpy as np

def make_gif(fold):
    fname="t{0:.2f}.jpg"
    times = np.hstack((np.arange(0,10,0.1), np.arange(10,11,0.1)))
    frames = []
    for t in times:
        frames.append(Image.open(fold+fname.format(t)))
    frame_one = frames[0]
    frame_one.save("NT.gif", format="GIF", append_images=frames, save_all=True, duration=1000, loop=0)


make_gif("GIF_IMAGES/")